# Ising Thermalization

A simulation of a thermalization process utilizing the Ising Model, and the Metropolis–Hastings Algorithm

# 1 Code

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

In [2]:
@njit
def generateSpins(L):
  return np.random.choice(np.array([1,-1]),(L,L))

In [3]:
@njit
def generateExp(T):
  b = 1/T
  return np.array([
                    1.,
                    np.exp((-b) * ( 4)),
                    np.exp((-b) * ( 8)),
                    np.exp((-b) * (-8)),
                    np.exp((-b) * (-4)),
                  ])

In [4]:
@njit
def energy(S):
  E = 0
  I, J = S.shape
  for i in range(I):
    for j in range(J):
      E -= S[i,j] * S[i,j-1]
      E -= S[i,j] * S[i-1,j]
  return E

@njit
def magnetization(S):
  return S.sum()

In [5]:
@njit
def deltaEnergy(spin, neighbors):
  return 2 * spin * neighbors.sum()

In [6]:
@njit
def accept(delta, exp):
  if np.random.random() < exp[int(delta/4)]:
    return True
  return False

In [7]:
@njit
def ising(L, T, max_steps=800):

  S = generateSpins(L)

  E = energy(S)
  M = magnetization(S)

  exp = generateExp(T)
  N = L**2

  states = np.zeros((max_steps+1, 2))
  states[0,0] = E
  states[0,1] = M

  # Metropolis
  for step in range(1, max_steps+1):
    for n in range(N):

      i, j = np.random.random(size=2) * L
      i = int(i)
      j = int(j)

      neighbors = np.array([S[i,j-1],
                            S[i-1,j],
                            S[i,(j+1)%L],
                            S[(i+1)%L,j]])

      delta = deltaEnergy(S[i,j], neighbors)
      if accept(delta, exp):
        S[i,j] *= -1
        E += delta
        M += 2 * S[i,j]

    states[step,0] = E
    states[step,1] = M
  return states

In [8]:
def compare(L, T, instances):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
  for i in range(instances):
    inst = ising(L,T)
    N, n = inst.shape

    ax1.plot(range(N), inst[:,0])
    ax2.plot(range(N), inst[:,1])

  ax1.set_xlabel('steps')
  ax1.set_ylabel('energy')
  ax2.set_xlabel('steps')
  ax2.set_ylabel('magnetization')

  fig.suptitle("N = " + str(L**2) + " , T = " + str(T))
  plt.show()

In [9]:
instances = 16
for l in [32, 64, 96]:
  for t in [.5, 1., 1.5, 2., 2.5, 3]:
    compare(l, t, instances)

Output hidden; open in https://colab.research.google.com to view.

# 2 Report

### Why do the possible values jump by 4?
- Because the energy generated by a spin is a function of the sum of its neighbors, making 4, 2, and 0 the only possible values. Since the delta will always be twice the original value generated (2x = x - (-x)), the only achievable values by changing a spin will be 8, 4 and 0.

### Why is the value +/- (2N-4) not attainable?
- Because the maximum and minimum energy values of the system depend on a perfect grid of spins, either all aligned or alternating in orientation. Any change to this grid will necessarily result in an energy delta of +/- 8.

### 2.1 Observations on the results
1. We can observe that, for a given temperature, the system's behavior is quite similar regardless of size.
2. As the tested temperature increased, there was a lower tendency for the system to remain magnetized, as well as a lower tendency to stay in the lowest energy state.

# 3 Values and Errors

Now that we already visualized the behavior across many sizes and temperatures, we will have a closer inspection in a particular configuration.

L = 32 was chosen since working with significantly larger cases would be too costly, which would limit the sample space if we wanted to repeat the experiment. The chosen temperature was T = 1.5K because it introduces some instability to the system while retaining the tendency toward stability. The total number of steps chosen was 5000, with 2000 for thermalization and 200 to calculate the 15 thermodynamic averages.

In [10]:
def MeanHeatInRange(E, b, N, m, i):
  start = int(i * m)
  end = int(start + m)
  return (b**2 / N) * (np.mean(E[start:end]**2) - np.mean(E[start:end])**2)

In [11]:
def MeanSusceptibilityInRange(M, b, N, m, i):
  start = int(i * m)
  end = int(start + m)
  return (b / N) * (np.mean(M[start:end]**2) - np.mean(M[start:end])**2)

In [12]:
def MeanHeat(E, b, N, n, m):

  array = []
  for i in range(n):
    array.append(MeanHeatInRange(E, b, N, m, i))

  array = np.array(array)
  mean = array.mean()

  return mean, array


In [13]:
def MeanSusceptibility(M, b, N, n, m):

  array = []
  for i in range(n):
    array.append(MeanSusceptibilityInRange(M, b, N, m, i))

  array = np.array(array)
  mean = array.mean()

  return mean, array

In [14]:
def MeanEnergy(E, n, m):

  array = []
  for i in range(n):
    start = int(i * m)
    end = int(start + m)
    array.append(E[start:end].mean())

  array = np.array(array)
  mean = array.mean()

  return mean, array

In [15]:
def MeanMagnetization(M, n, m):

  array = []
  for i in range(n):
    start = int(i * m)
    end = int(start + m)
    array.append(M[start:end].mean())

  array = np.array(array)
  mean = array.mean()
  return mean, array

In [16]:
def err(mean, array, n):
  return np.sqrt(((mean - array)**2).sum() / (n**2 - n))

In [17]:
L = 32
N = 5000
Nterm = 2000
Nmcs = N - Nterm
n = 5
m = Nmcs/n

rounding = 4

for T in [.5, 1, 1.5]:
  b = 1/T
  print("T = \t", T)

  states = ising(L, T, max_steps=N)

  E = states[Nterm:,0]
  M = states[Nterm:,1]

  Eµ, Earray = MeanEnergy(E, n, m)
  error = err(Eµ, Earray, n)
  print("Eµ = \t", round(Eµ, rounding), " \t +/- ", round(error,rounding))

  Mµ, Marray = MeanMagnetization(M, n, m)
  error = err(Mµ, Marray, n)
  print("Mµ = \t", round(Mµ, rounding), " \t +/- ", round(error,rounding))

  meanHeat, arrayHeat = MeanHeat(E, b, Nmcs, n, m)
  error = err(meanHeat, arrayHeat, n)
  print("Cµ = \t", round(meanHeat, rounding), " \t +/- ", round(error,rounding))

  meanSusceptibility, arraySusceptibility = MeanSusceptibility(M, b, Nmcs, n, m)
  error = err(meanSusceptibility, arraySusceptibility, n)
  print("Xµ = \t", round(meanSusceptibility,rounding), " \t +/- ", round(error, rounding))

  print()


T = 	 0.5
Eµ = 	 -2048.0  	 +/-  0.0
Mµ = 	 -1024.0  	 +/-  0.0
Cµ = 	 0.0  	 +/-  0.0
Xµ = 	 0.0  	 +/-  0.0

T = 	 1
Eµ = 	 -2045.1587  	 +/-  0.0909
Mµ = 	 -1023.272  	 +/-  0.0232
Cµ = 	 0.0073  	 +/-  0.0004
Xµ = 	 0.0005  	 +/-  0.0

T = 	 1.5
Eµ = 	 -1998.9853  	 +/-  0.3396
Mµ = 	 1010.47  	 +/-  0.0951
Cµ = 	 0.0674  	 +/-  0.0033
Xµ = 	 0.0091  	 +/-  0.0004

